In [ ]:
# Step 1: Install dependencies
!pip install -q transformers sentence-transformers pymupdf langchain faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.0 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import fitz
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load models
embedder = SentenceTransformer("all-MiniLM-L6-v2")
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Global state
chunks = []
index = None
original_chunks = []
chat_history = ""  # لحفظ سجل المحادثة

# Utils
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def chunk_text(text, chunk_size=300):
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

def build_faiss_index(chunks):
    embeddings = [embedder.encode(chunk) for chunk in chunks]
    dim = len(embeddings[0])
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    return index, embeddings

def load_pdf(file_path):
    global chunks, index, original_chunks, chat_history
    text = extract_text_from_pdf(file_path)
    chunks = chunk_text(text)
    index, _ = build_faiss_index(chunks)
    original_chunks = chunks
    chat_history = ""  # reset chat
    return "✅ Medical PDF loaded successfully!", ""

def chat_with_pdf(user_input):
    global chat_history
    if not index:
        return chat_history + "\n❌ Please upload a PDF first."

    q_emb = embedder.encode(user_input)
    D, I = index.search(np.array([q_emb]), k=3)
    top_chunks = [original_chunks[i] for i in I[0]]
    context = " ".join(top_chunks)
    result = qa_pipeline(question=user_input, context=context)
    answer = result['answer']

    # 👤 = شخص عادي
    chat_history += f"\n👤 You: {user_input}\n🤖 Assistant: {answer}\n\n"
    return chat_history

# Build UI
with gr.Blocks(css=".gr-button {background-color: #2563eb !important; color: white} .gr-markdown {font-size: 16px;}") as demo:
    gr.Markdown("""
    <div style="text-align:center; padding: 20px;">
        <h1 style="color:#1e3a8a;"> Smart Medical Assistant</h1>
        <p style="font-size:18px;">Upload a medical PDF and chat with it.</p>
    </div>
    """)

    with gr.Row():
        pdf_input = gr.File(type="filepath", label="📄 Upload Medical PDF", file_types=[".pdf"])
        status = gr.Textbox(label="🟢 Status", interactive=False)

    chatbox = gr.Textbox(label="🧾 Chat", lines=20, interactive=False, placeholder="Your conversation will appear here...")
    user_input = gr.Textbox(label="✏️ Type your question and press Enter", placeholder="e.g. What is the diagnosis?", lines=1)

    pdf_input.change(fn=load_pdf, inputs=pdf_input, outputs=[status, chatbox])
    user_input.submit(fn=chat_with_pdf, inputs=user_input, outputs=chatbox)

demo.launch(share=True)


Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://29c82aaacb1e08b95b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
